In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Analiza danych syntetycznych klientów (CSV) z użyciem NumPy, pandas, matplotlib.

Uruchomienie (w tym samym folderze co CSV):
    python analiza_klienci.py --csv dane_syntetyczne_faker.csv --out out

Wyniki:
- Konsola: podsumowania, tabele przestawne.
- Katalog --out: wykresy PNG i pliki CSV z agregacjami.
"""

'\nAnaliza danych syntetycznych klientów (CSV) z użyciem NumPy, pandas, matplotlib.\n\nUruchomienie (w tym samym folderze co CSV):\n    python analiza_klienci.py --csv dane_syntetyczne_faker.csv --out out\n\nWyniki:\n- Konsola: podsumowania, tabele przestawne.\n- Katalog --out: wykresy PNG i pliki CSV z agregacjami.\n'

In [2]:
import argparse
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
def load_data(csv_path: Path) -> pd.DataFrame:
    df = pd.read_csv(csv_path, encoding="utf-8")
    # sanity check i typy
    expected_cols = ["idklienta", "imię", "nazwisko", "miasto", "liczba_zamówień", "kategoria"]
    missing = [c for c in expected_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Brak kolumn w CSV: {missing}")
    df["idklienta"] = pd.to_numeric(df["idklienta"], errors="coerce").astype("Int64")
    df["liczba_zamówień"] = pd.to_numeric(df["liczba_zamówień"], errors="coerce")
    return df

In [4]:
def basic_overview(df: pd.DataFrame):
    print("\n=== PODSTAWOWY PRZEGLĄD ===")
    print("Liczba rekordów:", len(df))
    print("Unikalnych klientów (idklienta):", df["idklienta"].nunique())
    print("\nPrzykładowe wiersze:\n", df.head(5))
    print("\nBraki danych per kolumna:\n", df.isna().sum())
    print("\nStatystyki liczby zamówień:\n", df["liczba_zamówień"].describe())

In [5]:

def ensure_out_dir(out_dir: Path):
    out_dir.mkdir(parents=True, exist_ok=True)


def plot_and_save(fig, out_path: Path):
    fig.tight_layout()
    fig.savefig(out_path, dpi=160)
    plt.close(fig)

In [6]:
def analyze_categories(df: pd.DataFrame, out_dir: Path):
    print("\n=== KATEGORIE ===")
    cat_counts = df["kategoria"].value_counts(dropna=False).sort_index()
    cat_ratio = (cat_counts / len(df)).round(4)
    cat_summary = pd.DataFrame({"liczba": cat_counts, "udział": cat_ratio})
    print(cat_summary)

    # zapis
    cat_summary.to_csv(out_dir / "kategorie_podsumowanie.csv", encoding="utf-8")

    # wykres udziałów kategorii (bar)
    fig = plt.figure()
    cat_counts.plot(kind="bar")
    plt.title("Liczba klientów w kategoriach")
    plt.xlabel("Kategoria")
    plt.ylabel("Liczba klientów")
    plot_and_save(fig, out_dir / "kategorie_bar.png")

    # wykres udziału (pie)
    fig = plt.figure()
    plt.pie(cat_counts.values, labels=cat_counts.index, autopct="%1.1f%%", startangle=90)
    plt.title("Udział kategorii")
    plot_and_save(fig, out_dir / "kategorie_pie.png")


def analyze_orders(df: pd.DataFrame, out_dir: Path):
    print("\n=== LICZBA ZAMÓWIEŃ ===")
    orders = df["liczba_zamówień"].values
    print("Min/Max:", np.min(orders), np.max(orders))
    print("Średnia/Mediana/Std:", np.mean(orders).round(2), np.median(orders), np.std(orders).round(2))

    # histogram liczby zamówień
    fig = plt.figure()
    plt.hist(orders, bins=57)  # zakres 4-60 => około 57 wartości
    plt.title("Histogram liczby zamówień")
    plt.xlabel("Liczba zamówień")
    plt.ylabel("Liczba klientów")
    plot_and_save(fig, out_dir / "liczba_zamowien_hist.png")

    # boxplot
    fig = plt.figure()
    plt.boxplot(orders, vert=True, showmeans=True)
    plt.title("Boxplot liczby zamówień")
    plt.ylabel("Liczba zamówień")
    plot_and_save(fig, out_dir / "liczba_zamowien_box.png")

In [7]:
def analyze_cities(df: pd.DataFrame, out_dir: Path, top_n: int = 20):
    print("\n=== MIASTA ===")
    city_counts = df["miasto"].value_counts().head(top_n)
    print(f"TOP {top_n} miast:\n", city_counts)

    city_counts.to_csv(out_dir / "miasta_top.csv", encoding="utf-8")

    fig = plt.figure()
    city_counts.plot(kind="bar")
    plt.title(f"TOP {top_n} miast (liczba klientów)")
    plt.xlabel("Miasto")
    plt.ylabel("Liczba klientów")
    plt.xticks(rotation=45, ha="right")
    plot_and_save(fig, out_dir / "miasta_top_bar.png")


def analyze_names(df: pd.DataFrame, out_dir: Path, top_n: int = 20):
    print("\n=== IMIONA / NAZWISKA ===")
    first_names = df["imię"].value_counts().head(top_n)
    last_names = df["nazwisko"].value_counts().head(top_n)

    print(f"TOP {top_n} imion:\n", first_names)
    print(f"\nTOP {top_n} nazwisk:\n", last_names)

    first_names.to_csv(out_dir / "imiona_top.csv", encoding="utf-8")
    last_names.to_csv(out_dir / "nazwiska_top.csv", encoding="utf-8")

    fig = plt.figure()
    first_names.plot(kind="bar")
    plt.title(f"TOP {top_n} imion")
    plt.xlabel("Imię")
    plt.ylabel("Liczba klientów")
    plt.xticks(rotation=45, ha="right")
    plot_and_save(fig, out_dir / "imiona_top_bar.png")

    fig = plt.figure()
    last_names.plot(kind="bar")
    plt.title(f"TOP {top_n} nazwisk")
    plt.xlabel("Nazwisko")
    plt.ylabel("Liczba klientów")
    plt.xticks(rotation=45, ha="right")
    plot_and_save(fig, out_dir / "nazwiska_top_bar.png")

In [8]:
def analyze_category_vs_orders(df: pd.DataFrame, out_dir: Path):
    print("\n=== KATEGORIA vs. LICZBA ZAMÓWIEŃ ===")
    pivot = df.pivot_table(index="kategoria", values="liczba_zamówień",
                           aggfunc=["count", "mean", "median", "std"])
    pivot.columns = ["liczba_klientów", "średnia_zamówień", "mediana_zamówień", "std_zamówień"]
    print(pivot)

    pivot.to_csv(out_dir / "kategoria_vs_zamowienia.csv", encoding="utf-8")

    # średnia zamówień per kategoria
    fig = plt.figure()
    pivot["średnia_zamówień"].plot(kind="bar")
    plt.title("Średnia liczba zamówień w kategoriach")
    plt.xlabel("Kategoria")
    plt.ylabel("Średnia liczba zamówień")
    plot_and_save(fig, out_dir / "kategoria_srednia_zamowien_bar.png")


def analyze_city_vs_category(df: pd.DataFrame, out_dir: Path, top_n: int = 15):
    # bierzemy top miasta wg liczby klientów
    top_cities = df["miasto"].value_counts().head(top_n).index
    sub = df[df["miasto"].isin(top_cities)]
    crosstab = pd.crosstab(sub["miasto"], sub["kategoria"], normalize="index").round(3)
    print("\n=== STRUKTURA KATEGORII w TOP MIASTACH ===\n", crosstab)

    crosstab.to_csv(out_dir / "miasto_vs_kategoria_top.csv", encoding="utf-8")

    # wykres (każda kategoria jako słupek obok siebie per miasto)
    fig = plt.figure()
    ax = plt.gca()
    crosstab.plot(kind="bar", ax=ax)
    plt.title(f"Udział kategorii w TOP {top_n} miastach")
    plt.xlabel("Miasto")
    plt.ylabel("Udział w ramach miasta")
    plt.xticks(rotation=45, ha="right")
    plot_and_save(fig, out_dir / "miasto_vs_kategoria_top_bar.png")

In [9]:
def main_datalore(csv_path="dane_syntetyczne_faker.csv"):
    print("Wczytywanie danych z:", csv_path)
    df = load_data(csv_path)

    # podstawowe info
    basic_overview(df)

    # analizy
    analyze_categories(df, Path("."))       # zapis + wykres inline
    analyze_orders(df, Path("."))
    analyze_cities(df, Path("."), top_n=20)
    analyze_names(df, Path("."), top_n=20)
    analyze_category_vs_orders(df, Path("."))
    analyze_city_vs_category(df, Path("."), top_n=15)

    # zwróć kilka obiektów do dalszej pracy w Datalore
    results = {
        "df": df,
        "top_miasta": df["miasto"].value_counts().head(20),
        "top_imiona": df["imię"].value_counts().head(20),
        "top_nazwiska": df["nazwisko"].value_counts().head(20),
        "pivot_kategoria_vs_zamowienia": df.pivot_table(
            index="kategoria",
            values="liczba_zamówień",
            aggfunc=["count", "mean", "median", "std"]
        )
    }
    return results


# w notebooku wystarczy:
results = main_datalore("dane_syntetyczne_faker.csv")
results["pivot_kategoria_vs_zamowienia"]



Wczytywanie danych z: dane_syntetyczne_faker.csv

=== PODSTAWOWY PRZEGLĄD ===
Liczba rekordów: 130000
Unikalnych klientów (idklienta): 130000

Przykładowe wiersze:
    idklienta     imię     nazwisko                miasto  liczba_zamówień  \
0          1    Tymon  Andryszczyk               Siedlce               22   
1          2     Sara     Grabczak                Zamość               28   
2          3   Stefan       Pęciak               Skawina                5   
3          4  Norbert      Kozdroń             Sochaczew               39   
4          5      Iwo       Bielat  Czerwionka-Leszczyny               33   

  kategoria  
0  Standard  
1  Standard  
2  Standard  
3  Standard  
4  Standard  

Braki danych per kolumna:
 idklienta          0
imię               0
nazwisko           0
miasto             0
liczba_zamówień    0
kategoria          0
dtype: int64

Statystyki liczby zamówień:
 count    130000.000000
mean         31.912623
std          16.459473
min           4.000000

,count,mean,median,std
,liczba_zamówień,liczba_zamówień,liczba_zamówień,liczba_zamówień
kategoria,,,,
Extra,26170,31.873901,32.0,16.410393
Gold,16865,31.975393,32.0,16.465245
Standard,86965,31.912103,32.0,16.473247
